In [1]:
# import the module
import tweepy
from tw_keys import consumer_key, consumer_secret, access_token, access_token_secret
import json
import pandas as pd
from tqdm.notebook import tqdm
from geopy.geocoders import Nominatim
from pathlib import Path

In [7]:

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# set access to user's access key and access secret
auth.set_access_token(access_token, access_token_secret)

# calling the api
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True)




In [ ]:
df = pd.read_csv("./input/tweets_jan_minmonth5/BOT_GEO_BERT_Corona_JAN1_2023.csv")
df_done = pd.read_csv("./output/geoid_done.csv")
df_done2 = pd.read_csv("./output/geoid_done2.csv")
print(len(df), "starting df")
print(len(set(df.geo)), "total geo ids")
geo_place_id_COVID_df_03_list = set(df.geo) - set(df_done.x) - set(df_done2.x)
print(len(geo_place_id_COVID_df_03_list), "remain to be processed")
#no match for "3d1da0ef32c06f83"


In [ ]:
#Loop to get the places for every id.
with open('./output/geo_results_1635-1655_d.json', 'a') as json_data:
    #if var_exists == False:
    json_data.write("[")
    for geo_id in tqdm(geo_place_id_COVID_df_03_list):
        print(geo_id, "starting")
        places = api.geo_id(geo_id)
        print(places)
        json.dump(places, json_data)
        json_data.write(",")
        print(geo_id, "done")
    json_data.write("]")
    json_data.close()


# add "[]" in the file 
# & remove ","
#erease file when starting over "a" is appending!

### Add county name

In [42]:
df_county =  pd.read_json("Scripts - get merge external/external_data/geo_results.json")
df_county
#locations = df_county["full_name"]
locations = df_county["centroid"]

# Initialize geolocator with Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# Initialize lists to store county and state information
counties = []

In [45]:
# Loop through locations and retrieve county and state information
for location in locations:
    lon, lat = location
    try:
        location_info = geolocator.reverse((lat,lon), exactly_one = True)
        if location_info:
            counties.append(location_info.raw["display_name"])
        else:
            counties.append('NoInfo')
    except:
        counties.append('NoInfo')
